In [30]:
!pip install ipywidgets

from IPython.display import display, clear_output
!pip install --upgrade notebook ipywidgets

import ipywidgets as widgets
import tkinter as tk
from tkinter import ttk, messagebox
import numpy as np
widgets.IntSlider()
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import joblib
from sklearn.preprocessing import StandardScaler

  Using cached notebook-7.4.7-py3-none-any.whl.metadata (10 kB)
Using cached notebook-7.4.7-py3-none-any.whl (14.3 MB)
  Attempting uninstall: notebook
    Found existing installation: notebook 7.4.6
    Uninstalling notebook-7.4.6:
      Successfully uninstalled notebook-7.4.6


In [28]:
try:
    ridge_model = joblib.load("Ridge_model.pkl")
    model_exists = True
    print("✅ Ridge model loaded successfully")
except:
    ridge_model = None
    model_exists = False
    print("⚠️ Ridge model not found. Using demo calculation.")

# Model features (exactly as in your training)
numerical_features = [
    'Temperature', 'Humidity', 'SquareFootage', 'Occupancy', 
    'HVACUsage', 'LightingUsage', 'DayOfWeek', 
    'Holiday', 'Hour', 'Day', 'Month'
]

# Create scaler with typical values (since we don't have the original)
scaler = StandardScaler()
typical_data = pd.DataFrame({
    'Temperature': [22.0], 'Humidity': [50.0], 'SquareFootage': [1500],
    'Occupancy': [5], 'HVACUsage': [0.5], 'LightingUsage': [0.5],
    'DayOfWeek': [4], 'Holiday': [0.1], 'Hour': [12], 'Day': [15], 'Month': [6]
})
scaler.fit(typical_data)

# Mappings
day_mapping = {
    'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 
    'Friday': 5, 'Saturday': 6, 'Sunday': 7
}
holiday_mapping = {'No': 0, 'Yes': 1}
hvac_mapping = {'Off': 0, 'On': 1}
lighting_mapping = {'Off': 0, 'On': 1}

# Widgets
day_of_week = widgets.Dropdown(
    options=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    description="Day:"
)
holiday = widgets.Dropdown(options=['No', 'Yes'], description="Holiday:")
hour = widgets.IntSlider(
    value=12, min=0, max=23, step=1, description="Hour:"
)
day = widgets.IntSlider(
    value=15, min=1, max=31, step=1, description="Day:"
)
month = widgets.IntSlider(
    value=6, min=1, max=12, step=1, description="Month:"
)
temperature = widgets.FloatSlider(
    value=22.0, min=10, max=35, step=0.5, description="Temp (°C):"
)
humidity = widgets.FloatSlider(
    value=50.0, min=20, max=80, step=1.0, description="Humidity (%):"
)
square_footage = widgets.IntSlider(
    value=1500, min=500, max=5000, step=100, description="Area (ft²):"
)
occupancy = widgets.IntSlider(
    value=5, min=1, max=20, step=1, description="Occupancy:"
)
hvac_usage = widgets.Dropdown(options=['Off', 'On'], description="HVAC:")
lighting_usage = widgets.Dropdown(options=['Off', 'On'], description="Lighting:")

button = widgets.Button(description="🔮 Predict Energy", button_style="success")
output = widgets.Output()

def predict_callback(b):
    with output:
        clear_output()
        
        # Prepare input data using exact feature order
        input_dict = {}
        for feature in numerical_features:
            if feature == 'Temperature':
                input_dict[feature] = temperature.value
            elif feature == 'Humidity':
                input_dict[feature] = humidity.value
            elif feature == 'SquareFootage':
                input_dict[feature] = square_footage.value
            elif feature == 'Occupancy':
                input_dict[feature] = occupancy.value
            elif feature == 'HVACUsage':
                input_dict[feature] = hvac_mapping[hvac_usage.value]
            elif feature == 'LightingUsage':
                input_dict[feature] = lighting_mapping[lighting_usage.value]
            elif feature == 'DayOfWeek':
                input_dict[feature] = day_mapping[day_of_week.value]
            elif feature == 'Holiday':
                input_dict[feature] = holiday_mapping[holiday.value]
            elif feature == 'Hour':
                input_dict[feature] = hour.value
            elif feature == 'Day':
                input_dict[feature] = day.value
            elif feature == 'Month':
                input_dict[feature] = month.value
        
        input_data = pd.DataFrame([input_dict])[numerical_features]  # Ensure correct order
        
        if model_exists:
            # Use Ridge model
            input_scaled = scaler.transform(input_data)
            prediction = ridge_model.predict(input_scaled)[0]
            print(f"⚡ Predicted Energy Consumption: {prediction:.2f} kWh")
        else:
            # Demo calculation
            base = square_footage.value * 0.04
            temp_factor = abs(temperature.value - 22) * 2
            hvac_factor = 20 if hvac_usage.value == 'On' else 0
            lighting_factor = 8 if lighting_usage.value == 'On' else 0
            occupancy_factor = occupancy.value * 2
            
            prediction = base + temp_factor + hvac_factor + lighting_factor + occupancy_factor
            print(f"⚡ Demo Energy Consumption: {prediction:.2f} kWh")

button.on_click(predict_callback)

# Display interface
display(widgets.VBox([
    widgets.HBox([day_of_week, holiday]),
    widgets.HBox([hour, day, month]),
    temperature, humidity, square_footage, occupancy,
    widgets.HBox([hvac_usage, lighting_usage]),
    button, output
]))

✅ Ridge model loaded successfully
